In [3]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

closeDataSet = pd.DataFrame()

class ft:
    def __init__(self, ticker, start, end):
        self.ticker = ticker
        self.start = start
        self.end = end
        
    def getCloseData(self):
        return fdr.DataReader(self.ticker, self.start, self.end)['Close']
    
    def getDayReturn(closeDataSet):
        return (closeDataSet / closeDataSet.shift(1)).fillna(1) 

    def getCumulativeReturn(closeDataSet = None):
        return closeDataSet / closeDataSet.iloc[0]

    def getPortfolioResult(closeDataSet, weight=None):
        dayReturn = ft.getDayReturn(closeDataSet)

        cumulativeReturn = ft.getCumulativeReturn(closeDataSet)

        if not weight:
            weight = [1 / len(closeDataSet.columns)] * len(closeDataSet.columns)
        # 포트 누적 수익률
        portfolioCumulativeReturn = (weight * cumulativeReturn).sum(axis=1)
        # 포트 일별 수익률
        portfolioDayReturn = (portfolioCumulativeReturn / portfolioCumulativeReturn.shift(1)).fillna(1)

        return portfolioDayReturn, portfolioCumulativeReturn
    
    def getEvaluation(cumReturn):
        # CAGR
        cagr = round(cumReturn.iloc[-1] ** (252/len(cumReturn)), 5)
        # mdd
        dd = (cumReturn.cummax() - cumReturn) / cumReturn.cummax() * 100
        mdd = dd.max()

        print(f'cagr : {cagr}\nmdd: {mdd}')

        cumReturn = pd.DataFrame(cumReturn)
        cumReturn.columns = ['cumReturn']
        cumReturn['year'] = cumReturn.index.year
        cumReturn['month'] = cumReturn.index.month

        # 월별 데이터, 연별 데이터
        monthData = cumReturn.drop_duplicates(['year','month'], keep='last').copy()
        yearData = cumReturn.drop_duplicates('year', keep='last').copy()

        # 월간 수익률
        monthData['monthReturn'] = ((monthData['cumReturn'] / monthData['cumReturn'].shift(1).fillna(1)) - 1) * 100

        # 연간 수익률
        yearData['yearReturn'] = (yearData['cumReturn'] / yearData['cumReturn'].shift(1) - 1) * 100

        # 수익곡선, 낙폭시각화
        f,(ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(20, 10))
        cumReturn['cumReturn'].plot(ax=ax1)
        (-1*dd).plot(ax=ax2)
        ax1.set_title('Cumulative Return & Draw Down')

        # 연간, 월간 수익률 시각화
        monthPivot = monthData.pivot('year', 'month', 'monthReturn')
        yearPivot = yearData.pivot('year', 'month','yearReturn')

        f,(ax1, ax2) = plt.subplots(1, 2, sharey=True, gridspec_kw={'width_ratios': [1, 5]}, figsize=(20, 10))
        g1 = sns.heatmap(yearPivot, ax=ax1, annot=True, fmt='.2f', linewidths=.1, cmap='RdYlGn', center=0, cbar=False)
        g2 = sns.heatmap(monthPivot, ax=ax2, annot=True, fmt='.2f', linewidths=.1, cmap='RdYlGn', center=0, cbar=False)
        ax1.set_title('Annual Return')
        ax2.set_title('Monthly Return')

        return cagr, dd, mdd

    def getRebalancingDate(closeDataSet, period='month'):
        data = closeDataSet.copy()
        data = pd.DataFrame(data)
        data.index = pd.to_datetime(data.index)
        data['year'] = data.index.year
        data['month'] = data.index.month

        # 월말 리밸런싱
        if period == 'month':
            rebalancingDate = data.drop_duplicates(['year', 'month'], keep='last').index

        # 분기말 리밸런싱
        if period == 'quarter':
            quarter = [3,6,9,12]
            data = data.loc[data['month'].isin(quarter)]
            rebalancingDate = data.drop_duplicates(['year', 'month'], keep='last').index

        # 연말 리밸런싱
        if period == 'year':
            rebalancingDate = data.drop_duplicates(['year'], keep='last').index

        return rebalancingDate

    def getRebalancingPortfolioResult(closeDataSet, period='month', weightDf=None):
        rebalancingDate = getRebalancingDate(closeDataSet, period)

        # 자산별 비중, 기본값 : 동일비중
        if weightDf is None:
            weightDf = pd.DataFrame([[1/len(closeDataSet.columns)] * len(closeDataSet.columns)] * len(rebalancingDate),
                                   index = rebalancingDate,
                                   columns = closeDataSet.columns)

        portfolio = pd.DataFrame()

        totalAsset = 1
        start = rebalancingDate[0]

        for end in rebalancingDate[1:]:
            weight = weightDf.loc[start]
            priceData = closeDataSet.loc[start:end]
            cumReturn = ft.getCumulativeReturn(priceData)
            weightedCumReturn = weight * cumReturn
            netCumReturn = totalAsset * weightedCumReturn

            start = end # 갱신
            totalAsset = netCumReturn.iloc[-1].sum()
            portfolio = pd.concat([portfolio, netCumReturn])

        portfolio = portfolio.loc[~portfolio.index.duplicated(keep='last')]
        portfolioCumulativeReturn = portfolio.sum(axis=1)
        portfolioDayReturn = (portfolioCumulativeReturn / portfolioCumulativeReturn.shift(1)).fillna(1)

        return portfolioDayReturn, portfolioCumulativeReturn


    def getWeightByAvgMomentumScore(closeDataSet, n=12):
        avgMomentumScore = 0
        priceOnRebalDate = closeDataSet.loc[getRebalancingDate(closeDataSet)]

        # 1 ~ n 개월 모멘텀 스코어 합
        for i in range(1, n+1):
            avgMomentumScore = np.where(priceOnRebalDate / priceOnRebalDate.shift(i) > 1, 1, 0) + avgMomentumScore

        # 평모스 계산
        avgMomentumScore = pd.DataFrame(avgMomentumScore, index=priceOnRebalDate.index, columns = priceOnRebalDate.columns)
        avgMomentumScore = avgMomentumScore / n

        # 모멘텀 스코어에 따른 weight 계산
        weight = avgMomentumScore.divide(avgMomentumScore.sum(axis=1), axis=0).fillna(0)

        # 투자비중 '0'인 구간에서는 현금 보유
        weight['cash'] = np.where(weight.sum(axis=1) == 0, 1, 0)

        return weight, avgMomentumScore


    def getGTAAWeight(closeDataSet):
        gtaaCol = ['SPY', 'EFA', 'IEF', 'DBC', 'VNQ']
        gtaaData = closeDataSet[gtaaCol]

        # 모멘텀 스코어 계산
        ma200 = gtaaData.rolling(window=200).mean()
        momentum_score = (gtaaData / ma200) - 1
        momentum_score.dropna(inplace=True)

        # 리밸런싱 일자
        rebalDate = getRebalancingDate(momentum_score)

        # weightDF 계산하기
        gtaaWeightDf = pd.DataFrame(np.where(momentum_score > 0, 0.2, 0), index=momentum_score.index, columns = gtaaData.columns)
        gtaaWeightDf = gtaaWeightDf.loc[rebalDate]
        gtaaWeightDf['cash'] = 1 - gtaaWeightDf.sum(axis=1)

        # 현금 포함한 가격 데이터
        gtaaDataWithCash = gtaaData.copy()
        gtaaDataWithCash.loc[:,'cash'] = 1
        #gtaaDataWithCash = gtaaDataWithCash.loc[rebalDate]
        gtaaDataWithCash = gtaaDataWithCash.loc['2010-12-31':]

        return gtaaWeightDf

    def getFAAWeight(closeDataSet):
        faaCol = ['VTI','VEA','VWO','BND','SHY','DBC']
        faaData = closeDataSet[faaCol]

        # 리밸런싱 날짜
        faaRebalDate = getRebalancingDate(faaData)

        # 모멘텀 : 4개월 수익률
        faaDataOnRebalDate = faaData.loc[faaRebalDate]
        momentum4 = faaDataOnRebalDate / faaDataOnRebalDate.shift(4)
        momentum4.dropna(inplace=True)
        momentum4Score = momentum4.rank(method='max', axis=1, ascending=False)

        # 변동성 : 4개월 일일수익률 표준편차
        std4 = pd.DataFrame()
        # 상관성 : 4개월 하나의 자산과 다른 6개 자산간의 일일수익률 상관성
        corr4 = pd.DataFrame()
        reverseFaaRebalDate = faaRebalDate[::-1]

        for index, date in enumerate(reverseFaaRebalDate):
            if index >= len(reverseFaaRebalDate) - 4:
                break

            # 4개월 전 시점
            before4month = reverseFaaRebalDate[index + 4]

            # 일일수익률
            dayReturn = ft.getDayReturn(closeDataSet=faaData)

            # 4개월 일일수익률 표준편차
            std = dayReturn.loc[date : before4month : -1].std()
            std.name = date
            std4 = std4.append(std)

            # 4개월 일일수익률 상관성
            corr = (dayReturn.loc[date : before4month : -1].corr(method='pearson')).sum(axis=1) - 1
            corr.name = date
            corr4 = corr4.append(corr)

        # 변동성 순위 정하고, 인덱스 오름차순 정렬
        std4Score = std4.rank(method='first', axis=1, ascending=True)
        std4Score = std4Score.sort_index(ascending=True)

        # 상관성 순위 정하고, 인덱스 오름차순 정렬
        corr4Score = corr4.rank(method='first', axis=1, ascending=True)
        corr4Score = corr4Score.sort_index(ascending=True)


        # 가중평균
        totalScore = (momentum4Score + 0.5 * (std4Score + corr4Score)).rank(method='first', axis=1, ascending=True)

        # 랭킹 상위 3개 + 모멘텀 1 이상 추출
        weight = (totalScore <= 3) & (momentum4 >= 1)
        faaWeight = weight.replace(True, 1/3).replace(False, 0)
        faaWeight['cash'] = 1 - faaWeight.sum(axis=1)

        # 현금 포함된 자산 가격
        faaDataWithCash = faaData.loc['2010-12-31':].copy()
        faaDataWithCash.loc[:, 'cash'] = 1

        return faaWeight


    def getVAAWeight(closeDataSet):
        vaaCol = ['SPY','VEA','EEM','AGG','LQD','SHY', 'IEF']
        vaaAttack = ['SPY','VEA','EEM','AGG']
        vaaDefense = ['LQD','SHY', 'IEF']
        vaaData = closeDataSet[vaaCol]

        # 모멘텀 스코어 구하기
        rebalData = getRebalancingDate(vaaData)
        vaaDataOnRebalDate = vaaData.loc[rebalData]

        momentum1 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(1) - 1   # 1개월 모멘텀
        momentum3 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(3) - 1   # 3개월 모멘텀
        momentum6 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(6) - 1   # 6개월 모멘텀
        momentum12 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(12) - 1   # 12개월 모멘텀

        momentumScore = 12 * momentum1 + 4 * momentum3 + 2 * momentum6 + momentum12   # 모멘텀 스코어
        momentumScore.dropna(inplace=True)

        # 공격자산 모두 0 이상이면 가장 스코어 높은 자산에 몰빵
        isAttack = (momentumScore[vaaAttack] > 0).all(axis=1)
        vaaWeight = momentumScore.apply(applyGetVAAWeight, axis=1, args=(isAttack,))

        return vaaWeight


    def applyGetVAAWeight(row, isAttack):
        vaaAttack = ['SPY', 'VEA','EEM', 'AGG']
        vaaDefense = ['LQD', 'SHY', 'IEF']

        # print(row)
        # print(row.name)
        # print(row.index)

        # 공격자산을 선택하는 경우
        if isAttack[row.name]:              # isAttack 이 불린값이므로 True 인 것만 이 조건하에 실행
            # 공격자산 중 모멘텀 스코어 높은 종목에 몰빵
            result = pd.Series(row.index == row[vaaAttack].idxmax(), index = row.index, name=row.name).astype(int)  # 1행씩 실행
            return result

        # 수비자산 선택하는 경우
        result = pd.Series(row.index == row[vaaDefense].idxmax(), index=row.index, name=row.name).astype(int)
        return result

    # vaaWeight = getVAAWeight(closeDataSet)
    # vaaWeight = vaaWeight.loc['2010-12-31':]



    def getDAAWeight(closeDataSet):
        daaCol = ['SPY','IWM','QQQ','VGK','EWJ','VWO','VNQ','GSG','GLD','TLT','HYG','LQD','SHY','IEF','BND']
        daaAttack = ['SPY','IWM','QQQ','VGK','EWJ','VNQ','GSG','GLD','TLT','HYG','LQD']

        daaDefense = ['SHY','IEF','TLT']
        daaCanary = ['VWO','BND']

        daaData = closeDataSet[daaCol].copy()
        daaData.dropna(inplace=True)

        # 모멘텀 스코어 계산
        rebalData = getRebalancingDate(daaData)
        daaDataOnRebalDate = daaData.loc[rebalData]

        momentum1 = daaDataOnRebalDate / daaDataOnRebalDate.shift(1) - 1   # 1개월 모멘텀
        momentum3 = daaDataOnRebalDate / daaDataOnRebalDate.shift(3) - 1   # 3개월 모멘텀
        momentum6 = daaDataOnRebalDate / daaDataOnRebalDate.shift(6) - 1   # 6개월 모멘텀
        momentum12 = daaDataOnRebalDate / daaDataOnRebalDate.shift(12) - 1   # 12개월 모멘텀

        momentumScore = 12 * momentum1 + 4 * momentum3 + 2 * momentum6 + momentum12   # 모멘텀 스코어
        momentumScore.dropna(inplace=True)

        isAttack = (momentumScore[daaCanary] > 0).sum(axis=1)
        daaWeight = momentumScore.apply(applyGetDAAWeight, axis=1, args=(isAttack,))

        return daaWeight

    def applyGetDAAWeight(row, isAttack):
        daaCol = ['SPY','IWM','QQQ','VGK','EWJ','VWO','VNQ','GSG','GLD','TLT','HYG','LQD','SHY','IEF','BND']
        daaAttack = ['SPY','IWM','QQQ','VGK','EWJ','VNQ','GSG','GLD','TLT','HYG','LQD']

        # UST 데이터 시점 문제로 TLT로 대체
        daaDefense = ['SHY','IEF','TLT']
        daaCanary = ['VWO','BND']

        if isAttack[row.name] == 2:
                                # 카나리아 자산 2개 모두 0초과일때 >>> 공격자산 상위 2개에 몰빵
            top2 = row[daaAttack].nlargest(n=2).index
            result = pd.Series(row.index.isin(top2), index=row.index, name=row.name).astype(int).replace(1, 0.5)
            return result

    #     if isAttack[row.name] == 1:
    #                             # 카나리아 자산 1개만 0초과 >>> 공격자산 1개 + 방어자산 1개
    #         topAttack = row[daaAttack].idxmax()
    #         topDefense = row[daaDefense].idxmax()
    #         result = pd.Series(row.index.isin([topAttack, topDefense]), index=row.index, name=row.name).astype(int).replace(1, 0.5)
    #         return result

    #                             # 카나리아 자산이 모두 모멘텀스코어 0 이하
    #     return pd.Series(row.index == row[daaDefense].idxmax(), index=row.index, name=row.name).astype(int)

        if isAttack[row.name] == 1:
                                # 카나리아 자산 1개만 0초과 >>> 공격자산 1개 + 방어자산 1개
            topAttack = row[daaAttack].idxmax()
            topDefense = row[daaDefense].idxmax()
            result = pd.Series(row.index.isin([topAttack, topDefense]), index=row.index, name=row.name)


                                # 카나리아 자산이 모두 모멘텀스코어 0 이하
        return pd.Series(row.index == row[daaDefense].idxmax(), index=row.index, name=row.name).astype(int)

In [64]:
if __name__ == "__main__":

    # def getRebalancingPortfolioResult(closeDataSet, period='month', weightDf=None):
    rebalancingDate = ft.getRebalancingDate(closeDataSet)

    # 자산별 비중, 기본값 : 동일비중
    if weightDf is None:
        weightDf = pd.DataFrame([[1/len(closeDataSet.columns)] * len(closeDataSet.columns)] * len(rebalancingDate),
                               index = rebalancingDate,
                               columns = closeDataSet.columns)

    portfolio = pd.DataFrame()

    totalAsset = 1
    start = rebalancingDate[0]

    for end in rebalancingDate[1:]:
        weight = weightDf.loc[start]
        priceData = closeDataSet.loc[start:end]
        cumReturn = ft.getCumulativeReturn(priceData)
        weightedCumReturn = weight * cumReturn
        netCumReturn = totalAsset * weightedCumReturn

        start = end # 갱신
        totalAsset = netCumReturn.iloc[-1].sum()
        portfolio = pd.concat([portfolio, netCumReturn])

    portfolio = portfolio.loc[~portfolio.index.duplicated(keep='last')]
    portfolioCumulativeReturn = portfolio.sum(axis=1)
    portfolioDayReturn = (portfolioCumulativeReturn / portfolioCumulativeReturn.shift(1)).fillna(1)

    # return portfolioDayReturn, portfolioCumulativeReturn